In [12]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
!git clone "https://github.com/ScierKnave/honor_project.git"
import requests
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ScierKnave/TorchMPS.git to /tmp/pip-req-build-nrzwb0kf
  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git /tmp/pip-req-build-nrzwb0kf
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py) ... done
fatal: destination path 'honor_project' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [13]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Student hyperparameters
# MPS parameters
bond_dim_HP = 40
# USING CUSTOM FEATURE MAP WITH FORK?: YES

# Training parameters
nepochs_student_HP = 30 
student_lr_HP = 1e-4
student_reg_HP = 0
student_loss_HP = nn.CrossEntropyLoss()


# Premilinaries: Importing the data and utils subroutines

In [14]:

# Import the *common* mnist train set and create a batch iterator for it.
train_set = torch.load('/content/honor_project/src/datasets/train_mnist.pt')
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_train_HP)),
    batch_size=batch_sz_HP
    )

# Import the mnist *common* test set and create a batch iterator for it.
test_set = torch.load('/content/honor_project/src/datasets/test_mnist.pt')
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_test_HP)),
    batch_size = batch_sz_HP
    )

In [15]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator, dataset_size):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / dataset_size # divide by total size

# Training the student model

In [16]:
def train_a_student():
  '''
  Trains a student model.
  '''

  # Initialize the MPS module
  student = MPS(
      input_dim = 28 ** 2,
      output_dim = 10,
      bond_dim = bond_dim_HP
  ).to(chosen_device) 
  #student.register_feature_map(feature_map_HP)

  # Instantiate the optimizer and softmax
  student_optimizer = torch.optim.Adam(
      student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
  )

  # Used on the inputs before the loss function
  LogSoftmax = nn.LogSoftmax(dim=1)

  # Create an array to store the val loss
  # of the student at each epoch
  stud_test_loss = np.array([])
  stud_train_loss = np.array([])

  softmax = nn.Softmax(dim=1)

  stud_train_loss = np.array([])
  stud_test_loss = np.array([])

  for epoch in range(nepochs_student_HP):
      '''
      Training loop through the epochs.
      '''
      for (x_mb, y_mb) in train_iterator:
          # Flatten the MNIST images, which come in matrix form
          x_mb = x_mb.reshape(-1, 784).to(chosen_device)
          y_mb = y_mb.to(chosen_device)

          student_logits = student(x_mb) 

          # Backpropagation
          loss = student_loss_HP( student_logits, y_mb)

          student_optimizer.zero_grad()
          loss.backward()
          student_optimizer.step()

      # Get accuracy over all test and training data for current epoch
      train_current_accuracy = round( get_acc(student, train_iterator, nb_train_HP).item(), 4)
      test_current_accuracy = round( get_acc(student, test_iterator, nb_test_HP).item(), 4)
      stud_train_loss = np.append(stud_train_loss, train_current_accuracy)
      stud_test_loss = np.append(stud_test_loss, test_current_accuracy)
  return(stud_train_loss, stud_test_loss)


# Repeat the training process in order to get the variance
global_stud_test_loss = np.array([])
global_stud_train_loss = np.array([])
for i in range(20):
  print("\n Training and testing of model ", i+1, " in progress...")
  (stud_train_loss, stud_test_loss) = train_a_student()
  print("Train loss: ", stud_train_loss)
  print("Test loss: ", stud_test_loss)
  if (i == 0):
    global_stud_train_loss = stud_train_loss
    global_stud_test_loss = stud_test_loss
  else:
    global_stud_train_loss = np.vstack((global_stud_train_loss, stud_train_loss))
    global_stud_test_loss = np.vstack((global_stud_test_loss, stud_test_loss))




 Training and testing of model  1  in progress...
Train loss:  [0.1481 0.1389 0.3382 0.6006 0.7125 0.7932 0.8477 0.8837 0.9032 0.9301
 0.9143 0.9317 0.9652 0.9581 0.977  0.9772 0.981  0.9734 0.9746 0.9871
 0.975  0.9864 0.9862 0.9872 0.996  0.9794 0.9611 0.9524 0.9852 0.9863]
Test loss:  [0.1472 0.1284 0.3076 0.6027 0.7243 0.8244 0.8385 0.8714 0.8796 0.9007
 0.8944 0.8884 0.91   0.9075 0.9205 0.8954 0.9226 0.9022 0.8999 0.9263
 0.9081 0.9184 0.917  0.9144 0.9189 0.8992 0.8887 0.8868 0.9111 0.9226]

 Training and testing of model  2  in progress...
Train loss:  [0.2061 0.1365 0.3225 0.7054 0.7895 0.8342 0.855  0.8933 0.9072 0.9408
 0.932  0.9568 0.9609 0.959  0.9751 0.9632 0.9688 0.9604 0.9809 0.9717
 0.9733 0.9831 0.9878 0.9833 0.9719 0.994  0.9746 0.9813 0.9637 0.9754]
Test loss:  [0.2227 0.1272 0.3234 0.7257 0.7848 0.8302 0.8568 0.8536 0.8824 0.9123
 0.8741 0.8983 0.8932 0.8977 0.9123 0.9024 0.9141 0.9097 0.9281 0.9091
 0.9064 0.9147 0.9141 0.9152 0.9007 0.917  0.9044 0.8975 0.8953 

In [17]:
# Print the final results and save them for the report.
print("Final results")
print("Train loss: ", global_stud_train_loss)
print("Test loss: ", global_stud_test_loss)

np.save('20x_mps_trainloss_40bd', stud_train_loss)
np.save('20x_mps_testloss_40bd', stud_test_loss)


Final results
Train loss:  [[0.1481 0.1389 0.3382 0.6006 0.7125 0.7932 0.8477 0.8837 0.9032 0.9301
  0.9143 0.9317 0.9652 0.9581 0.977  0.9772 0.981  0.9734 0.9746 0.9871
  0.975  0.9864 0.9862 0.9872 0.996  0.9794 0.9611 0.9524 0.9852 0.9863]
 [0.2061 0.1365 0.3225 0.7054 0.7895 0.8342 0.855  0.8933 0.9072 0.9408
  0.932  0.9568 0.9609 0.959  0.9751 0.9632 0.9688 0.9604 0.9809 0.9717
  0.9733 0.9831 0.9878 0.9833 0.9719 0.994  0.9746 0.9813 0.9637 0.9754]
 [0.1928 0.1071 0.1    0.1    0.1065 0.1708 0.1588 0.2945 0.6212 0.7548
  0.8431 0.8829 0.8782 0.8909 0.91   0.9424 0.9274 0.9303 0.9744 0.9612
  0.9716 0.9739 0.988  0.9843 0.9655 0.9873 0.9741 0.9861 0.9837 0.9922]
 [0.2611 0.2254 0.17   0.2824 0.2648 0.2146 0.1613 0.1273 0.1403 0.4043
  0.5996 0.7474 0.8589 0.8246 0.8929 0.9032 0.9307 0.9503 0.9351 0.9636
  0.9709 0.964  0.9426 0.9538 0.966  0.9742 0.9716 0.972  0.9466 0.9853]
 [0.1894 0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1    0.1
  0.3803 0.6167 0.7735 0.8221 0.8899